In [ ]:
import pandas as pd
import numpy as np
import math

from datetime import datetime
from datetime import timedelta

import os
import time
from timeit import default_timer as timer

from IPython.display import clear_output

pd.set_option('display.max_rows', 500)

In [ ]:
df = pd.read_csv("datahistorica122019.csv", encoding='utf8')

df

In [ ]:
df["estacion"] = df["estacion"].replace("AgÃ\x83Â¼ero", "Agüero")
df["estacion"] = df["estacion"].replace("AgÃ¼ero", "Agüero")
df["estacion"] = df["estacion"].replace("Saenz PeÃ\x83Â±a ", "Saenz Peña")
df["estacion"] = df["estacion"].replace("Saenz PeÃ±a ", "Saenz Peña")
df["estacion"] = df["estacion"].replace("Saenz Peña ", "Saenz Peña")
df["linea"] = df["linea"].replace(["LineaA", "LineaB", "LineaC", "LineaD", "LineaE", "LineaH"
                                  ], ["A", "B", "C", "D", "E", "H"])

In [ ]:
from datetime import datetime

dias_semana = {"0": "lunes", "1": "martes", "2": "miercoles", "3": "jueves", 
               "4": "viernes", "5": "sabado", "6": "domingo"}

df["dia_semana"] = df["fecha"]

def dia_semana(fecha):
    feriados = ["2019-01-01", "2019-03-04", "2019-03-05", "2019-03-24", "2019-03-04",
               "2019-04-02", "2019-04-19", "2019-05-01", "2019-05-25", "2019-07-08",
               "2019-07-09", "2019-08-19", "2019-10-14", "2019-03-04", "2019-12-08",
               "2019-12-25"]
    if fecha in feriados:
        return "feriado"
    dia = datetime.strptime(fecha, "%Y-%m-%d").weekday()
    return dias_semana[f"{dia}"]

for dia in df["dia_semana"].unique():
    print(dia)
    df["dia_semana"] = df["dia_semana"].replace(dia, dia_semana(dia))
    
df["dia_semana"].unique()

In [ ]:
def main(dia_semana):
    t0 = timer()
    meses = ["201901", "201902", "201903", "201904", "201905", "201906", "201907", 
                 "201908", "201909", "201910", "201911", "201912"]

    lineas = {"A": "lightblue", "B": "red", "C": "blue", "D": "green", "E": "purple", "H": "yellow"}
    
    def promediar_ingresos(dia_semana):

        lineas_lista = []
        estaciones_lista = []
        meses_lista = []
        horarios_lista = []
        promedio_lista = []

        for linea in ["A"]:
            df_linea = df[df["linea"]==linea]
            estaciones = df_linea["estacion"].unique()
            for estacion in estaciones:
                df_estacion = df_linea[df_linea["estacion"]==estacion]
                for mes in meses:
                    df_mes = df_estacion.query(f"periodo == {mes}")
                    horarios = sorted(df_mes.query(f"dia_semana == '{dia_semana}'")["desde"].unique())
                    for horario in horarios:
                        # por alguna razon hay horarios de entrada donde el subte no funciona, puede que sean los trabajadores
                        if horario in ['04:45:00', '05:00:00',
                                      '04:30:00', '02:45:00',
                                       '03:45:00', '01:00:00', '02:30:00', '01:15:00', '02:15:00',
                                       '00:15:00', '04:00:00', '04:15:00', '02:00:00', '00:30:00',
                                       '00:45:00', '01:30:00', '01:45:00', '03:00:00', '03:15:00',
                                       '03:30:00']:
                            continue
                        promedio = df_mes.query(f"dia_semana == '{dia_semana}'" 
                                                         ).query(f"desde == '{horario}'")["total"].sum()/len(
                                                        df_mes.query(f"dia_semana == '{dia_semana}'"
                                                         )["fecha"].unique())
                        # se le suma 0.5 al promedio para redonder para arriba o para abajo
                        promedio = math.floor(promedio+0.5)
                        lineas_lista.append(linea)
                        estaciones_lista.append(estacion)
                        meses_lista.append(mes)
                        horarios_lista.append(horario)
                        promedio_lista.append(promedio)

        diccionario = {"linea": lineas_lista, "estacion": estaciones_lista, "mes": meses_lista, 
                       "horario": horarios_lista, "ingresos": promedio_lista}
        dataframe_prueba = pd.DataFrame.from_dict(data=diccionario)
        return dataframe_prueba
    
    df2 = promediar_ingresos(dia_semana)

    # promediar viajes segun sentido en estas estaciones que actuan como indicadores

    def promediar_sentido(dia_semana):
        estaciones_sentido = ["Acoyte", "Rio de Janeiro", "Castro Barros", "Loria", "Congreso", "Saenz Peña", "Piedras"]

        lineas_lista = []
        estaciones_lista = []
        meses_lista = []
        horarios_lista = []
        molinete_lista = []
        promedio_lista = []

        for estacion in estaciones_sentido:
            df_estacion = df[df["estacion"]==estacion]
            for mes in meses:
                df_mes = df_estacion.query(f"periodo == {mes}")
                horarios = sorted(df_mes.query(f"dia_semana == '{dia_semana}'"
                                                     )["desde"].unique())
                for horario in horarios:
                    # por alguna razon hay horarios de entrada donde el subte no funciona, puede que sean los trabajadores
                    if horario in ['04:45:00', '05:00:00',
                                  '04:30:00', '02:45:00',
                                   '03:45:00', '01:00:00', '02:30:00', '01:15:00', '02:15:00',
                                   '00:15:00', '04:00:00', '04:15:00', '02:00:00', '00:30:00',
                                   '00:45:00', '01:30:00', '01:45:00', '03:00:00', '03:15:00',
                                   '03:30:00']:
                        continue
                    else:
                        for molinete in df_mes.query(f"dia_semana == '{dia_semana}'"
                                                             ).query(f"desde == '{horario}'")["molinete"].unique():
                            promedio = df_mes.query("dia_semana == 'lunes'"
                                                             ).query(f"desde == '{horario}' & molinete == '{molinete}'"
                                                                    )["total"].sum()/len(
                                                            df_mes.query(f"dia_semana == '{dia_semana}'"
                                                             )["fecha"].unique())
                            promedio = math.floor(promedio+0.5)
                            lineas_lista.append("A")
                            estaciones_lista.append(estacion)
                            meses_lista.append(mes)
                            horarios_lista.append(horario)
                            molinete_lista.append(molinete)
                            promedio_lista.append(promedio)

        diccionario = {"linea": lineas_lista, "estacion": estaciones_lista, "mes": meses_lista, 
                       "horario": horarios_lista, "molinete": molinete_lista, "ingresos": promedio_lista}
        dataframe_sentido_A = pd.DataFrame.from_dict(data=diccionario)
        return dataframe_sentido_A
    
    dataframe_sentido_A = promediar_sentido(dia_semana)
    
    lineas_lista = []
    estaciones_lista = []
    meses_lista = []
    horarios_lista = []
    N_lista = []
    N_por_lista = []
    S_lista = []
    S_por_lista = []
    NAN_lista = []
    
    meses = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 
                 201908, 201909, 201910, 201911, 201912]
    
    for estacion in ["San Pedrito", "Flores", "Carabobo", "Puan", "Primera Junta", "Acoyte", "Rio de Janeiro",
                 "Castro Barros", "Loria", "Plaza Miserere", "Alberti", "Pasco", "Congreso", "Saenz Peña", "Lima", 
                 "Piedras", "Peru", "Plaza de Mayo"]:
        if estacion in ("Alberti", "Pasco", "Plaza de Mayo", "San Pedrito"):
            df_estacion = df2[df2["estacion"]==estacion]
            for mes in meses:
                df_mes = df_estacion.query(f"mes == '{mes}'")
                horarios = df_mes["horario"].unique()
                for horario in horarios:
                    if estacion in ("Pasco", "San Pedrito"):
                        lineas_lista.append("A")
                        estaciones_lista.append(estacion)
                        meses_lista.append(mes)
                        horarios_lista.append(horario)
                        N_lista.append(int(df_mes.query(f"horario == '{horario}'")["ingresos"]))
                        S_lista.append(0)
                        N_por_lista.append(1)
                        S_por_lista.append(0)
                        NAN_lista.append(0)
                    else:
                        lineas_lista.append("A")
                        estaciones_lista.append(estacion)
                        meses_lista.append(mes)
                        horarios_lista.append(horario)
                        N_lista.append(0)
                        S_lista.append(int(df_mes.query(f"horario == '{horario}'")["ingresos"]))
                        N_por_lista.append(0)
                        S_por_lista.append(1)
                        NAN_lista.append(0)
        elif estacion in ("Flores", "Carabobo", "Puan", "Primera Junta", "Plaza Miserere", "Lima", "Peru"):
            df_estacion = df2[df2["estacion"]==estacion]
            for mes in meses:
                df_mes = df_estacion.query(f"mes == '{mes}'")
                horarios = df_mes["horario"].unique()
                for horario in horarios:
                    lineas_lista.append("A")
                    estaciones_lista.append(estacion)
                    meses_lista.append(mes)
                    horarios_lista.append(horario)
                    N_lista.append(0)
                    S_lista.append(0)
                    N_por_lista.append(np.nan)
                    S_por_lista.append(np.nan)
                    NAN_lista.append(int(df_mes[df_mes["horario"]==horario]["ingresos"]))
        else:
            df_estacion = dataframe_sentido_A[dataframe_sentido_A["estacion"]==estacion]
            for mes in meses:
                if estacion == "Peru" and mes == "201912":
                    df_mes = df_estacion.query(f"mes == '{mes}'")
                    horarios = df_mes["horario"].unique()
                    for horario in horarios:
                        lineas_lista.append("A")
                        estaciones_lista.append(estacion)
                        meses_lista.append(mes)
                        horarios_lista.append(horario)
                        N_lista.append(0)
                        S_lista.append(int(df_mes.query(f"horario == '{horario}'")["ingresos"]))
                        N_por_lista.append(0)
                        S_por_lista.append(1)
                        NAN_lista.append(int(df_mes[df_mes["horario"]==horario]["ingresos"]))
                else:
                    df_mes = df_estacion.query(f"mes == '{mes}'")
                    horarios = df_mes["horario"].unique()
                    for horario in horarios:
                        df_horario = df_mes[df_mes["horario"]==horario]
                        N = []
                        S = []
                        NAN = []
                        for molinete in df_horario["molinete"].unique():
                            if "_N_" in molinete:
                                N.append(int(df_horario[df_horario["molinete"]==molinete]["ingresos"]))
                            elif "_S_" in molinete:
                                S.append(int(df_horario[df_horario["molinete"]==molinete]["ingresos"]))
                            else:
                                NAN.append(int(df_horario[df_horario["molinete"]==molinete]["ingresos"]))
                        lineas_lista.append("A")
                        estaciones_lista.append(estacion)
                        meses_lista.append(mes)
                        horarios_lista.append(horario)
                        N_lista.append(math.floor(sum(N)+0.5)) if len(N) != 0 else N_lista.append(0)
                        S_lista.append(math.floor(sum(S)+0.5)) if len(S) != 0 else S_lista.append(0)
                        N_por_lista.append(sum(N)/(sum(N)+sum(S))) if (sum(N)+sum(S))>0 else N_por_lista.append(0)
                        S_por_lista.append(sum(S)/(sum(N)+sum(S))) if (sum(N)+sum(S))>0 else S_por_lista.append(0)
                        NAN_lista.append(math.floor(sum(NAN)+0.5)) if len(NAN) != 0 else NAN_lista.append(0)


    diccionario = {"linea": lineas_lista, "estacion": estaciones_lista, "mes": meses_lista, 
                   "horario": horarios_lista, "sentido_mayo": N_lista, "sentido_mayo_por": N_por_lista,
                   "sentido_pedrito": S_lista, "sentido_pedrito_por": S_por_lista, "sentido_nan": NAN_lista}
    dataframe_sentido_A_parcial = pd.DataFrame.from_dict(data=diccionario)

    df_vacia = dataframe_sentido_A_parcial[dataframe_sentido_A_parcial["mes"]=="2020"]
    
    for mes in meses:
        df_mes = dataframe_sentido_A_parcial[dataframe_sentido_A_parcial["mes"]==mes]
        horarios = df_mes["horario"].unique()
        for horario in horarios:
            df_horario = df_mes[df_mes["horario"]==horario]
            df_vacia = pd.concat([df_vacia, df_horario])
    
    # aca tuve que crear otro df sin las estaciones alberti y pasco, que me arruinan la interpolación,
    # y usar esa df para agregar los valores a la df original con todas las estaciones

    df_vacia_2 = dataframe_sentido_A_parcial[dataframe_sentido_A_parcial["mes"]=="2020"]

    for mes in meses:
        df_mes = df_vacia[df_vacia["mes"]==mes]
        horarios = df_mes["horario"].unique()
        for horario in horarios:
            df_horario = df_mes[df_mes["horario"]==horario]
            df_horario_2 = df_horario.query("estacion != 'Alberti' & estacion != 'Pasco'")
            df_horario_2 = df_horario_2[["sentido_mayo_por", "sentido_pedrito_por"]].interpolate()

            estaciones_interpolar = ["Flores", "Carabobo", "Puan", "Primera Junta", "Plaza Miserere", "Lima", "Peru"]

            for estacion in estaciones_interpolar:
                for index, row in df_horario.iterrows():
                    if row["estacion"]==estacion:
                        try:
                            df_horario.at[index, "sentido_mayo_por"] = df_horario_2[df_horario_2.index==index]["sentido_mayo_por"]
                            df_horario.at[index, "sentido_pedrito_por"] = df_horario_2[df_horario_2.index==index]["sentido_pedrito_por"]
                            df_horario.at[index, "sentido_mayo"] = np.floor(df_horario[df_horario.index==index]["sentido_nan"]*df_horario[df_horario.index==index]["sentido_mayo_por"]+0.5)
                            df_horario.at[index, "sentido_pedrito"] = np.floor(df_horario[df_horario.index==index]["sentido_nan"]*df_horario[df_horario.index==index]["sentido_pedrito_por"]+0.5)
                        except:
                            pass
            df_vacia_2 = pd.concat([df_vacia_2, df_horario])

    dataframe_sentido_A_final = df_vacia_2

    df_a = dataframe_sentido_A_final

    meses = list(df_a["mes"].unique())
    estaciones = df_a["estacion"].unique()

    df_vacia = df_a[df_a["mes"]=="2020"].drop(columns=df_a.columns[-1])

    for mes in meses:
        df_a_mes = df_a[df_a["mes"]==mes]
        for estacion in estaciones:
            df_a_estacion = df_a_mes[df_a_mes["estacion"]==estacion]
            for index, row in df_a_estacion.iterrows():
                horario = row["horario"]
                horarios = []
                N_lista = []
                N_por_lista = []
                S_lista = []
                S_por_lista = []
                for i in range (0, 15):
                    hora = datetime.strptime(horario, "%H:%M:%S") + timedelta(minutes=i)
                    hora = hora.strftime('%H:%M:%S')
                    horarios.append(hora)
                    if i != 0:
                        N_lista.append(np.nan)
                        N_por_lista.append(np.nan)
                        S_lista.append(np.nan)
                        S_por_lista.append(np.nan)
                    else:
                        N_lista.append(row["sentido_mayo"])
                        N_por_lista.append(row["sentido_mayo_por"])
                        S_lista.append(row["sentido_pedrito"])
                        S_por_lista.append(row["sentido_pedrito_por"])
                diccionario = {"linea": "A", "estacion": estacion, "mes": mes, 
                       "horario": horarios, "sentido_mayo": N_lista, "sentido_mayo_por": N_por_lista,
                       "sentido_pedrito": S_lista, "sentido_pedrito_por": S_por_lista}
                df_cuarto = pd.DataFrame(data=diccionario)
                df_vacia = df_vacia.append(df_cuarto)

    df_aux = df_vacia[["sentido_mayo", "sentido_mayo_por", "sentido_pedrito", 
                       "sentido_pedrito_por"]].interpolate()
    df_vacia[["sentido_mayo", "sentido_mayo_por", 
              "sentido_pedrito", "sentido_pedrito_por"]] = df_aux[["sentido_mayo", "sentido_mayo_por", 
                                        "sentido_pedrito", "sentido_pedrito_por"]]
    df_vacia[["sentido_mayo", "sentido_pedrito"]] = df_vacia[["sentido_mayo", "sentido_pedrito"]]/15
    
    df_a_final = df_vacia
    df_a_final.to_csv(f"sentido_viajes_A_x_min_{dia_semana}.csv", index=False)
    t1 = timer()
    print("Interpolacion para:", dia_semana, "completada en:", round(t1-t0, 2), "s")

In [ ]:
main("lunes")
main("martes")
main("miercoles")
main("jueves")
main("viernes")
main("sabado")
main("domingo")
main("feriado")